# Experiment 01-02: OpenCL Brute

Test if the OpenCL Brute SHA256 kernel is loading.

In [89]:
import sys
from glob import glob
import numpy as np
import pyopencl

## Helper Functions

In [90]:
def get_first_platform_by_name(name : str) -> pyopencl.Platform:
    """
    Get the first platform with the given name.

    Parameters
    ----------
    name : str
        The name of the platform to get.
    """
    for platform in pyopencl.get_platforms():
        if platform.name == name:
            return platform

    raise ValueError(f"No platform named '{name}'")

In [91]:
def build_program(program_files : list, context : pyopencl.Context,
        build_options=[]) -> pyopencl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    context : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source_code = ''

    for ocl_file in program_files:
        with open(ocl_file, 'r') as opencl_file:
            file_source_code = opencl_file.read()
            program_source_code += '\n' + file_source_code

    program_source = pyopencl.Program(context, program_source_code)
    program = program_source.build(options=build_options)
            
    return program

In [92]:
def run_ocl_kernel(queue, kernel, global_size,
                   input_tuples, output_tuples,
                   local_size = (32,)):
    
    # copying data onto the device
    for (value, buffer) in input_tuples:
        pyopencl.enqueue_copy(queue, src=value, dest=buffer)
    
    # running program on the device
    kernel_arguments  = [buffer for (_,buffer) in input_tuples] 
    kernel_arguments += [buffer for (_,buffer) in output_tuples]
        
    kernel(queue, global_size, local_size, *kernel_arguments)

    # copying data off the device
    for (value, buffer) in output_tuples:
        pyopencl.enqueue_copy(queue, src=buffer, dest=value)
        
    # waiting for everything to finish
    queue.finish()

## Setup the device and context

In [93]:
ocl_platforms = (platform.name 
                 for platform in pyopencl.get_platforms())
print("\n".join(ocl_platforms))

Apple


In [94]:
gpu_platform = get_first_platform_by_name("Apple")
gpu_devices = gpu_platform.get_devices(device_type=pyopencl.device_type.GPU)
gpu_context = pyopencl.Context(devices=gpu_devices)

print(gpu_devices)
print(gpu_context)

[<pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>]
<pyopencl.Context at 0x600001b380b0 on <pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>>


## Load Hashcat SHA256 kernel

In [95]:
# get a list of the source files
program_files = glob('../opencl_brute/*.cl')
program_files = [
    '../opencl_brute/Library/worker/generic/buffer_structs_template.cl',
    '../opencl_brute/Library/worker/generic/sha256.cl',
]
print(program_files)

['../opencl_brute/Library/worker/generic/buffer_structs_template.cl', '../opencl_brute/Library/worker/generic/sha256.cl']


In [96]:
# build the program
build_options = [
    # '-I', '../hashcat/include/',
    # '-I', '../hashcat/OpenCL/',
]

gpu_program = build_program(program_files, gpu_context, build_options)

print(gpu_program)

RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>:

Compilation failed: 

program_source:39:5: error: invalid token at start of a preprocessor expression
#if wordSize == 4
    ^
program_source:20:18: note: expanded from macro 'wordSize'
#define wordSize <word_size>
                 ^
program_source:49:7: error: invalid token at start of a preprocessor expression
#elif wordSize == 8
      ^
program_source:20:18: note: expanded from macro 'wordSize'
#define wordSize <word_size>
                 ^
program_source:74:5: error: unknown type name 'word'
    word length; // in bytes
    ^
program_source:75:5: error: unknown type name 'word'
    word buffer[inBufferSize];
    ^
program_source:75:17: error: expected expression
    word buffer[inBufferSize];
                ^
program_source:23:30: note: expanded from macro 'inBufferS